# MLT Lab 2

In [31]:
import pandas as pd
import numpy as np
import re
import nltk
import collections
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to /home/dormant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/dormant/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dormant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Task 1: most common words in chapters

In [2]:
with open('./alice.txt') as f:
    l = f.readlines()
    
l = [item for item in l if item != '\n']
    
splitter = ['CHAPTER I.\n', 'CHAPTER II.\n', 'CHAPTER III.\n',
            'CHAPTER IV.\n', 'CHAPTER V.\n', 'CHAPTER VI.\n',
            'CHAPTER VII.\n', 'CHAPTER VIII.\n', 'CHAPTER IX.\n',
            'CHAPTER X.\n', 'CHAPTER XI.\n', 'CHAPTER XII.\n',
            '*** END OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***\n']
splitter_idx = [l.index(item) for item in splitter]

alice = [0] * (len(splitter) - 1)

for i in range(len(splitter)-1):
    alice[i] = l[splitter_idx[i]:splitter_idx[i+1]]

alice = [" ".join(item) for item in alice]
alice = [re.sub(r"[‘’.*?,!“”—;()]", " ", item) for item in alice]
alice = [re.sub("\n|_", " ", item) for item in alice]
alice = [re.sub(" +", " ", item) for item in alice]
alice = [item.strip() for item in alice]

# Tokenization
alice = [item.lower() for item in alice]
alice = [item.split(" ") for item in alice]

# Stop-words and lemmatization
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

for i in range(len(alice)):
    alice[i] = [item for item in alice[i] if item not in stop_words]
    alice[i] = [lemmatizer.lemmatize(item) for item in alice[i]]
    
alice = [" ".join(item).strip() for item in alice]

In [3]:
alice[-1]

'chapter xii alice evidence cried alice quite forgetting flurry moment large grown last minute jumped hurry tipped jury-box edge skirt upsetting juryman head crowd lay sprawling reminding much globe goldfish accidentally upset week oh beg pardon exclaimed tone great dismay began picking quickly could accident goldfish kept running head vague sort idea must collected put back jury-box would die trial cannot proceed said king grave voice juryman back proper place repeated great emphasis looking hard alice said alice looked jury-box saw haste put lizard head downwards poor little thing waving tail melancholy way quite unable move soon got put right signifies much said think would quite much use trial one way soon jury little recovered shock upset slate pencil found handed back set work diligently write history accident except lizard seemed much overcome anything sit mouth open gazing roof court know business king said alice nothing said alice nothing whatever persisted king nothing whatev

In [4]:
vect_bow = CountVectorizer()
vect_bow.fit(alice)

simple_train_dtm = vect_bow.transform(alice)
df = pd.DataFrame(simple_train_dtm.toarray(), columns=vect_bow.get_feature_names_out()).T
df = df.drop(['alice'])

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
abide,0,0,0,0,0,1,0,0,0,0,0,0
able,0,1,0,0,0,0,0,0,0,0,0,0
absence,0,0,0,0,0,0,0,0,1,0,0,0
absurd,0,0,1,0,0,1,0,0,0,0,0,0
acceptance,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
top_ten = [df[col].nlargest(10).index.values for col in df.columns]

In [7]:
for num, item in enumerate(top_ten):
    print(splitter[num][:-2])
    print(", ".join(item))
    print()

CHAPTER I
little, way, like, think, see, door, one, rabbit, time, could

CHAPTER II
mouse, little, oh, said, dear, go, thing, foot, like, must

CHAPTER III
said, mouse, dodo, know, one, soon, bird, course, dry, long

CHAPTER IV
little, rabbit, one, bill, said, get, heard, sure, thought, voice

CHAPTER V
said, caterpillar, pigeon, serpent, little, well, know, minute, one, size

CHAPTER VI
said, cat, like, footman, duchess, little, baby, mad, much, know

CHAPTER VII
said, hatter, dormouse, march, hare, time, well, know, thing, tea

CHAPTER VIII
said, queen, head, king, cat, three, hedgehog, like, one, went

CHAPTER IX
said, turtle, mock, gryphon, duchess, queen, went, never, little, say

CHAPTER X
said, gryphon, turtle, mock, would, lobster, dance, soup, beautiful, voice

CHAPTER XI
said, king, hatter, court, dormouse, one, witness, queen, began, rabbit

CHAPTER XII
said, king, jury, would, little, queen, know, head, one, rabbit



## Task 2: most common verbs in sentences with Alice

In [50]:
alice = [0] * (len(splitter) - 1)

for i in range(len(splitter)-1):
    alice[i] = " ".join(l[splitter_idx[i]+2:splitter_idx[i+1]])

alice = re.split(r'[.!?]', " ".join(alice))
alice = [re.sub(r"[‘’*,“”—;()]", " ", item) for item in alice]
alice = [re.sub("\n|_", " ", item) for item in alice]
alice = [re.sub(" +", " ", item) for item in alice]
alice = [item.strip() for item in alice]

# Tokenization
alice = [item.lower() for item in alice]
alice = [item.split(" ") for item in alice]

# Stop-words and lemmatization
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

for i in range(len(alice)):
    alice[i] = [item for item in alice[i] if item not in stop_words]
    alice[i] = [lemmatizer.lemmatize(item) for item in alice[i]]

# Sentences, that contain Alice
alice = [" ".join(item).strip() for item in alice]
alice = [item for item in alice if "alice" in item]

# Verbs in those sentences
for i in range(len(alice)):
    alice[i] = [item for item in alice[i].split(" ") if 'VB' in nltk.pos_tag([item])[0][1]]

alice = [" ".join(item).strip() for item in alice if len(item)>0]
alice = " ".join(alice).split(" ")
print(alice)

['beginning', 'get', 'tired', 'sitting', 'say', 'occurred', 'wondered', 'seemed', 'took', 'looked', 'hurried', 'started', 'flashed', 'seen', 'take', 'see', 'went', 'considering', 'get', 'went', 'stopping', 'falling', 'tumbling', 'let', 'see', 'showing', 'say', 'got', 'say', 'began', 'talking', 'began', 'get', 'went', 'saying', 'looked', 'hurrying', 'went', 'say', 'turned', 'getting', 'locked', 'trying', 'walked', 'wondering', 'get', 'came', 'made', 'opened', 'led', 'looked', 'longed', 'get', 'get', 'go', 'see', 'happened', 'begun', 'seemed', 'waiting', 'went', 'hoping', 'find', 'shutting', 'said', 'printed', 'say', 'going', 'ventured', 'finding', 'buttered', 'finished', 'said', 'shutting', 'waited', 'see', 'going', 'know', 'said', 'going', 'finding', 'happened', 'decided', 'going', 'said', 'leave', 'said', 'make', 'make', 'get', 'holding', 'growing', 'find', 'remained', 'got', 'expecting', 'happen', 'seemed', 'go', 'cried', 'go', 'ashamed', 'said', 'say', 'go', 'came', 'began', 'starte

In [51]:
counter = collections.Counter(alice)
print("Most common verbs in sentences with Alice:")
for i in counter.most_common(10):
    print(f"\t{i[0]}\t{i[1]}")

Most common verbs in sentences with Alice:
	said	174
	went	30
	know	28
	looked	24
	say	22
	see	22
	get	20
	began	20
	got	18
	go	18


In [59]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='english')

# if we stem verbs, we get:
alice = [snowball.stem(item) for item in alice]
counter = collections.Counter(alice)
print("Most common verb stems in sentences with Alice:")
for i in counter.most_common(10):
    print(f"\t{i[0]}\t{i[1]}")

Most common verb stems in sentences with Alice:
	said	174
	look	39
	went	30
	go	30
	know	30
	get	29
	say	25
	see	22
	began	20
	got	18
